In [1]:
import pandas as pd
import json
import re

# map the title of books and the reviews

In [2]:
# extract the title of books
bk_2019 = pd.read_csv("../data/raw_data/books_2019_df.csv")
title_2019 = bk_2019['title'].apply(lambda x: x.lower())
title_df_2019 = pd.DataFrame(title_2019,columns=['title'])
title_df_2019['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2019) + 1)]
title_df_2019.head(3)
title_df_2019.shape

(400, 2)

In [3]:
bk_2020 = pd.read_csv("../data/raw_data/books_2020_df.csv")
title_2020 = bk_2020['title'].apply(lambda x: x.lower())
title_df_2020 = pd.DataFrame(title_2020,columns=['title'])
title_df_2020['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2020) + 1)]
title_df_2020.shape

(399, 2)

In [4]:
bk_2021 = pd.read_csv("../data/raw_data/books_2021_df.csv")
title_2021 = bk_2021['title'].apply(lambda x: x.lower())
title_df_2021 = pd.DataFrame(title_2021,columns=['title'])
title_df_2021['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2021) + 1)]
title_df_2021.shape

(340, 2)

In [5]:
bk_2022 = pd.read_csv("../data/raw_data/books_2022_df.csv")
title_2022 = bk_2022['title'].apply(lambda x: x.lower())
title_df_2022 = pd.DataFrame(title_2022,columns=['title'])
title_df_2022['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2022) + 1)]
title_df_2022.shape

(340, 2)

In [6]:
bk_2023 = pd.read_csv("../data/raw_data/books_2023_df.csv")
title_2023 = bk_2023['title'].apply(lambda x: x.lower())
title_df_2023 = pd.DataFrame(title_2023,columns=['title'])
title_df_2023['title_id'] = ['Book_' + str(i) for i in range(1, len(title_df_2023) + 1)]
title_df_2023.shape

(299, 2)

In [7]:
with open("../data/raw_data/reviews_2019.json", 'r',encoding='utf-8') as file:
    review_2019 = json.load(file)

with open("../data/raw_data/reviews_2020.json", 'r',encoding='utf-8') as file:
    review_2020 = json.load(file)

with open("../data/raw_data/reviews_2021.json", 'r',encoding='utf-8') as file:
    review_2021 = json.load(file)

with open("../data/raw_data/reviews_2022.json", 'r',encoding='utf-8') as file:
    review_2022 = json.load(file)

with open("../data/raw_data/reviews_2023.json", 'r',encoding='utf-8') as file:
    review_2023 = json.load(file)

In [8]:
# define function 

def review_to_df(data):
    reviews = []

    # Iterate through each book and its reviews
    for book_id, reviews_dict in data.items():
        for review_id, review_content in reviews_dict.items():
            # Extracting the reviewer's name and review text
            # Here we assume the name ends at the first digit, which is extremely simplistic
            reviewer_name = ''.join([char for char in review_content if not char.isdigit()]).split(',')[0]
            review_text = review_content.split('followers')[-1]  # This is a simplistic split

            # Append the extracted information as a dictionary to the list
            reviews.append({
                'title_id': book_id,
                'review_id': review_id,
                'review_text': review_text.strip()
            })
    df_reviews = pd.DataFrame(reviews)
    return df_reviews

In [31]:
review_df_2019 = review_to_df(review_2019)
review_df_2019.shape

(3980, 3)

In [24]:
review_df_2020 = review_to_df(review_2020)
review_df_2021 = review_to_df(review_2021)
review_df_2022 = review_to_df(review_2022)
review_df_2023 = review_to_df(review_2023)

In [18]:
review_df_2021.head()

,title_id,review_id,review_text
0,Book_1,1,"August 10, 2022“Purposeful awkwardness” is how..."
1,Book_1,2,"September 11, 2021Let me start this review by ..."
2,Book_1,3,"December 14, 2022Well, well, well.Look what th..."
3,Book_1,4,"September 14, 2021I am writing a Sally Rooney ..."
4,Book_1,5,"September 16, 2021I don't know what it is.All ..."


In [23]:
# Regular expression to match the date format
# Define the date pattern
date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}'

# Function to extract and remove dates
def extract_and_remove_date(text):
    match = re.search(date_pattern, text)
    if match:
        date = match.group(0)  # Extract the date
        cleaned_text = re.sub(date_pattern, '', text)  # Remove the date from the text
        return date, cleaned_text
    return None, text  # Return None for date and original text if no date found

In [32]:
# Apply the function to each entry in the review_text column
review_df_2019[['date', 'review_text']] = review_df_2019['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2019 = pd.merge(title_df_2019, review_df_2019, on='title_id', how='left')
df_2019 = df_2019[['title','review_id','date','review_text']]
df_2019.head(10)

,title,review_id,date,review_text
0,the testaments,1,"May 29, 2020",I guess I'll have to be the one who says what ...
1,the testaments,2,"September 13, 2019",I can sum it up simply:this book is not needed...
2,the testaments,3,"January 22, 2020",Return to GileadCheck your expectations at the...
3,the testaments,4,"August 15, 2021",Winner of best fiction category but it’s not m...
4,the testaments,5,"December 4, 2019",A review in 5 words:Unnecessary. Pointless. Ru...
5,the testaments,6,"October 23, 2019",ReadI liked this one more than A Handmaid's Ta...
6,the testaments,7,"October 9, 2019",I haven't even finished season 2 of the tv sho...
7,the testaments,8,"September 14, 2019",This was my most anticipated book for 2019.Wai...
8,the testaments,9,"September 22, 2019","""How tedious is a tyranny in the throes of ena..."
9,the testaments,10,"September 18, 2019",It's not easy being the most anticipated book ...


In [25]:
review_df_2020[['date', 'review_text']] = review_df_2020['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2020 = pd.merge(title_df_2020, review_df_2020, on='title_id', how='left')
df_2020 = df_2020[['title','review_id','date','review_text']]
df_2020

,title,review_id,date,review_text
0,the midnight library,1,"January 21, 2022",I liked this book until it suddenly decided to...
1,the midnight library,2,"November 24, 2020","“The only way to learn is to live”Fuck, this b..."
2,the midnight library,3,"February 15, 2023",Everybody probably knows the premise of this b...
3,the midnight library,4,"September 14, 2023",Okay! No more words! This is one of the best s...
4,the midnight library,5,"May 31, 2020",It is no secret that Matt Haig has mental heal...
...,...,...,...,...
3967,wild symphony,6,"November 15, 2020",What a fun way to interest young ones (and the...
3968,wild symphony,7,"November 12, 2020",Dan Brown es conocido por “El código Da vinci”...
3969,wild symphony,8,"October 11, 2021","I liked this book a lot... very interactive, w..."
3970,wild symphony,9,"July 22, 2022",Labai nuostabios iliustracijos. Tokios-mano mė...


In [26]:
review_df_2021[['date', 'review_text']] = review_df_2021['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2021 = pd.merge(title_df_2021, review_df_2021, on='title_id', how='left')
df_2021 = df_2021[['title','review_id','date','review_text']]
df_2021

,title,review_id,date,review_text
0,"beautiful world, where are you",1,"August 10, 2022",“Purposeful awkwardness” is how I can describe...
1,"beautiful world, where are you",2,"September 11, 2021",Let me start this review by saying that I thin...
2,"beautiful world, where are you",3,"December 14, 2022","Well, well, well.Look what the cat dragged in...."
3,"beautiful world, where are you",4,"September 14, 2021",I am writing a Sally Rooney novel. I am typing...
4,"beautiful world, where are you",5,"September 16, 2021",I don't know what it is.All of you scratching ...
...,...,...,...,...
3377,where's my joey?,6,"December 31, 2020",I won this through Goodread's FirstReads givea...
3378,where's my joey?,7,"September 22, 2021",This is a sweet adorable book that teaches chi...
3379,where's my joey?,8,"January 27, 2021",Where's My Joey? by Wendy Monica Winter is a c...
3380,where's my joey?,9,"March 9, 2021",Where’s My Joey? by Wendy Monica Winter is a c...


In [27]:
review_df_2022[['date', 'review_text']] = review_df_2022['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2022 = pd.merge(title_df_2022, review_df_2022, on='title_id', how='left')
df_2022 = df_2022[['title','review_id','date','review_text']]
df_2022

,title,review_id,date,review_text
0,"tomorrow, and tomorrow, and tomorrow",1,"June 25, 2022",Katherine337 reviewsThis book is so utterly pr...
1,"tomorrow, and tomorrow, and tomorrow",2,"July 16, 2022","Tomorrow, and Tomorrow, and Tomorrow I will st..."
2,"tomorrow, and tomorrow, and tomorrow",3,"December 14, 2022",***5 Stars***This was one of those books where...
3,"tomorrow, and tomorrow, and tomorrow",4,"May 11, 2022","Complicated and MemorableTomorrow, and Tomorro..."
4,"tomorrow, and tomorrow, and tomorrow",5,"July 13, 2022",I feel... really disappointed by this. I've lo...
...,...,...,...,...
3377,how to heal a gryphon,6,"September 25, 2023",Remember the classicStrega NonabyTomie dePaola...
3378,how to heal a gryphon,7,"September 26, 2022",How To Heal A Gryphon was a pretty good Middle...
3379,how to heal a gryphon,8,"March 28, 2022",With Christmas Eve rapidly approaching where 1...
3380,how to heal a gryphon,9,"September 23, 2022",Rachael173 reviews3.5 stars rounded upGiada Be...


In [28]:
review_df_2023[['date', 'review_text']] = review_df_2023['review_text'].apply(
    lambda x: pd.Series(extract_and_remove_date(x))
)
df_2023 = pd.merge(title_df_2023, review_df_2023, on='title_id', how='left')
df_2023 = df_2023[['title','review_id','date','review_text']]
df_2023

,title,review_id,date,review_text
0,yellowface,1,"January 12, 2024",2.5/5i feel weird writing this review. for one...
1,yellowface,2,"March 24, 2024",‘Don’t ghosts just want to be remembered?’A bo...
2,yellowface,3,"January 23, 2023","lmao.the frustration, as always, is that rf ku..."
3,yellowface,4,"July 11, 2023",Whew child. Some of these reviews are doing ex...
4,yellowface,5,"January 25, 2024","If publishing is rigged, you might as well mak..."
...,...,...,...,...
2985,"not funny: essays on life, comedy, culture, et...",6,"November 23, 2023",DNF / abandoned p 76. Average comedian memoir-...
2986,"not funny: essays on life, comedy, culture, et...",7,"December 23, 2022",If you pick up Academy Award nominee Jena Frie...
2987,"not funny: essays on life, comedy, culture, et...",8,"May 4, 2023",Sharon L5 reviewsIf she was trying to be funny...
2988,"not funny: essays on life, comedy, culture, et...",9,"August 11, 2023",Laura Hobbs18 reviewsNot funny. While I starte...


In [29]:
df = pd.concat([df_2019,df_2020,df_2021,df_2022,df_2023], ignore_index=True)
df = df.drop(columns=['date'])
df.head(3)

,title,review_id,review_text
0,the testaments,1,I guess I'll have to be the one who says what ...
1,the testaments,2,I can sum it up simply:this book is not needed...
2,the testaments,3,Return to GileadCheck your expectations at the...


In [70]:
df.shape

(17708, 3)

In [71]:
df.isna().sum()

title          0
review_id      8
review_text    8
dtype: int64

In [75]:
df.fillna(value="Not found", inplace=True)

In [80]:
df['title'].nunique()

1687

In [81]:
# convert the reviews in 10 columns
columns = {"title": []}

# Initialize review columns
for i in range(1, 11):
    columns[f"review{i}"] = []

# Dictionary to hold review text for each book
review_dict = {}

# Iterate through the data and populate the review dictionary
for index, row in df.iterrows():
    title = row["title"]
    review_id = row["review_id"]
    review_text = row["review_text"]
    if title not in review_dict:
        review_dict[title] = {}
    review_dict[title][f"review{review_id}"] = review_text

# Iterate through the review dictionary and populate the column dictionaries
for title, reviews in review_dict.items():
    columns["title"].append(title)
    # Fill in review columns
    for i in range(1, 11):
        columns[f"review{i}"].append(reviews.get(f"review{i}", None))
    
# Convert dictionaries to pandas DataFrame for easier manipulation
new_df = pd.DataFrame(columns)

In [82]:
new_df

,title,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10
0,the testaments,I guess I'll have to be the one who says what ...,I can sum it up simply:this book is not needed...,Return to GileadCheck your expectations at the...,Winner of best fiction category but it’s not m...,A review in 5 words:Unnecessary. Pointless. Ru...,ReadI liked this one more than A Handmaid's Ta...,I haven't even finished season 2 of the tv sho...,This was my most anticipated book for 2019.Wai...,"""How tedious is a tyranny in the throes of ena...",It's not easy being the most anticipated book ...
1,normal people,“Life offers up these moments of joy despite e...,Disclaimer: I understand thatNormal Peopleis v...,No one can be independent of other people comp...,Sally Rooney is the real dealNormal Peoplehas ...,I am not sure how to write this review because...,Normal Peoplestarts out beautifully. Marianne ...,Man booker prize long list nominee and Costa b...,"wow. one of the most frustrating, but humanisi...",Whoever was responsible for the marketing of t...,The book aims to hit emotional poignancy by sh...
2,where the forest meets the stars,I think so far this is one of the best book I’...,This was an amazon first read selection for Fe...,"There are books you read that you enjoy, there...",| |blog|tumblr|ko-fi| |If I could just rate th...,what an absolutely delightful surprise gem of ...,There’s a reason (or two or three) why Where t...,2020 F.A.B. Bookclub pick # I.❤️. F.A.B.fab-bo...,I’ve been wanting to read this novel for awhil...,"Ok, I see all the glowing reviews and think th...","4.5 or maybe 10 stars, I don't know. My mind i..."
3,"ask again, yes","“The thing is, Peter, grown-ups don’t know wha...",There is a soul-baring emotional honesty toAsk...,5 Stars.Edited after much thought to revise ra...,* I've added a link to an article called A con...,"Four angsty, dysfunctional, mind blowing, push...","suburban drama!! family dynamics!!truly, what'...","""The thing is, Peter, grown-ups don't know wha...",4.5 starsI have always enjoyed reading family ...,Unresolved childhood experiences of loss and t...,***NOW AVAILABLE***This is one of those books ...
4,queenie,"He put a hand on my thigh and moved it higher,...",This is the kind of novel whose excellence sne...,I empathized a lot with the protagonist and he...,Queenie is the newest debut sensation coming o...,"5 shiny, rebellious, beautiful stars!As soon a...",I don’t know where to start with this book. It...,This book is being pitches as Bridget Jones Di...,A tiresome novel that made me cranky. Everythi...,Caution: Rated R for adult content 🔥🔥🔥This boo...,ReadWhy I love itby Jojo MoyesI have to confes...
...,...,...,...,...,...,...,...,...,...,...,...
1682,misfit: growing up awkward in the '80s,Mary521 reviewsIt cannot be overstated how muc...,Shawn Robare165 reviewsMy wife introduced me t...,"This has a happy ending, a long-term remission...",Rob Cusick38 reviewsA beautifully written book...,I first saw Gary Gulman on late night Comedy C...,Lindsay Vlasak210 reviewsI’ve loved Gary Gulma...,This is a reflective memoir by comedian Gary G...,I listened to this book and loved it so much. ...,4.5 starsI picked this up because I like Gulma...,"I didn’t think it was that funny, but it was i..."
1683,"unreliable narrator: me, myself, and impostor ...",3.5 stars rounded up because I think the marke...,adele107 reviews1 followerMuch more serious th...,“Feel free to add your own laugh track”Whew th...,Aparna Nancherla is a smart writer & brings he...,Aparna shines in her personal narrative. She i...,"Honest and straightforward, alternately funny ...","Witty, smart, well cited, really thoughtful qu...","It might be an issue with my expectations, but...",3.75 ⭐️2 likesLikeComment,Not at all what I expected - certainly a marke...
1684,america the beautiful?: one woman in a borrowe...,Katrina Margolis118 reviewsSimultaneously made...,"Jacie Tengesdal40 reviewsA funny, feminist tak..."

In [83]:
new_df.fillna(value="Not found", inplace=True)

In [88]:
new_df = new_df[new_df['title']!="title not found"]

In [89]:
new_df['title'].nunique()

1686

In [90]:
# export as csv file
new_df.to_csv('../data/database/reviews.csv', index=False)